In [9]:
%reload_ext autoreload
%autoreload 2

In [40]:
import errno
import os
import random
import shutil

from fastai.text import * 

# Data structuring

First we need to split our data up into training and validation sets (TODO: Strip out duplicates - different keys is ok though)

In [25]:
DATA_DIR_ALL = Path('data/encoded')

In [26]:
OUT_DIR = Path('data/split')

In [41]:
def ensure_dir_exists(path):
    """ Ensure that a directory at the given path exists """
    try:
        os.makedirs(path)
    except OSError as e:
        if e.errno != errno.EEXIST:
            raise

In [31]:
all_fnames = [encoded for encoded in os.listdir(DATA_DIR_ALL)]
RANDOM_SEED = 42
random.Random(RANDOM_SEED).shuffle(all_fnames)

In [37]:
training_set = all_fnames[: int(len(all_fnames) * 0.8)]
validation_set = all_fnames[len(training_set):]

In [42]:
for set_dir, data in zip(('train', 'valid'), (training_set, validation_set)):
    ensure_dir_exists(OUT_DIR/set_dir)
    for fname in data:
        shutil.copy(DATA_DIR_ALL/fname, OUT_DIR/set_dir)

# Model training

In [43]:
DATA_DIR = OUT_DIR

In [44]:
VOCAB = [f'S-{i}' for i in range(24, 76)] + ['R']
len(VOCAB)

53

We have a very limited 'vocabulary' of musical tokens. Tokenization simply involves splitting the comma-delimited list of strings.

In [69]:
class Tokenizer(object):
    @staticmethod
    def process_all(txt_batch):
        return [txt.split(',') for txt in txt_batch]

In [70]:
data_lm = TextLMDataBunch.from_folder(DATA_DIR, tokenizer=Tokenizer, include_bos=False, include_eos=False)

In [72]:
data_lm.save('data_lm_export.pkl')

In [74]:
data_lm = load_data(DATA_DIR, 'data_lm_export.pkl')

In [76]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5, pretrained=False)
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time


KeyboardInterrupt: 